In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

In [28]:
# แก้ไข cell ที่โหลดข้อมูล

# โหลดข้อมูล
df = pd.read_csv('../../data/baseline.csv')

# แปลง date column เป็น datetime และตั้งเป็น index
df['วันที่'] = pd.to_datetime(df['วันที่'])
df = df.set_index('วันที่')

print("Available columns:", df.columns.tolist())
print("Data shape:", df.shape)

# ✅ แก้ไข: ทำความสะอาดข้อมูลให้เป็นตัวเลขทั้งหมด
exclude_columns = ['rain_average', 'day_type']
train_lines = []

for col in df.columns:
    if col not in exclude_columns:
        # ตรวจสอบและทำความสะอาดข้อมูล
        print(f"\nProcessing column: {col}")
        print(f"Original dtype: {df[col].dtype}")
        print(f"Unique values (first 10): {df[col].unique()[:10]}")
        
        # แทนที่ 'not available' และ string อื่นๆ ด้วย NaN
        df[col] = df[col].replace(['not available', 'N/A', '', ' '], np.nan)
        
        # แปลงเป็นตัวเลข
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # ตรวจสอบจำนวนข้อมูลที่ valid
        valid_count = df[col].notna().sum()
        total_count = len(df)
        
        print(f"Valid data points: {valid_count}/{total_count} ({valid_count/total_count*100:.1f}%)")
        
        # เก็บเฉพาะ column ที่มีข้อมูล valid อย่างน้อย 50%
        if valid_count >= total_count * 0.5:
            train_lines.append(col)
            print(f"✅ Added {col} to training list")
        else:
            print(f"❌ Skipped {col} - insufficient valid data")

print(f"\nFinal train lines: {train_lines}")

# แบ่งข้อมูลเป็นชุดเทรนและชุดทดสอบ (30 วันสุดท้าย)
split_idx = len(df) - 30
train_df = df.iloc[:split_idx].copy()
test_df = df.iloc[split_idx:].copy()

print(f"\nTraining data: {len(train_df)} days")
print(f"Test data: {len(test_df)} days")
print(f"Train period: {train_df.index[0]} to {train_df.index[-1]}")
print(f"Test period: {test_df.index[0]} to {test_df.index[-1]}")

Available columns: ['day_name', 'day_type', 'is_holiday', 'holiday_subject', 'รถไฟฟ้า ARL', 'รถไฟฟ้า BTS', 'รถไฟฟ้าสายสีชมพู', 'รถไฟฟ้าสายสีน้ำเงิน', 'รถไฟฟ้าสายสีม่วง', 'รถไฟฟ้าสายสีเหลือง', 'รถไฟฟ้าสายสีแดง', 'rain_average']
Data shape: (2073, 12)

Processing column: day_name
Original dtype: object
Unique values (first 10): ['วันพุธ' 'วันพหัสบดี' 'วันศุกร์' 'วันเสาร์' 'วันอาทิตย์' 'วันจันทร์'
 'วันอังคาร']
Valid data points: 0/2073 (0.0%)
❌ Skipped day_name - insufficient valid data

Processing column: is_holiday
Original dtype: object
Unique values (first 10): ['holiday' 'normal' 'festival']
Valid data points: 0/2073 (0.0%)
❌ Skipped is_holiday - insufficient valid data

Processing column: holiday_subject
Original dtype: object
Unique values (first 10): ['วันขึ้นปีใหม่' 'normal' 'วันเด็ก' 'วันมาฆบูชา' 'วันหยุดชดเชย'
 'วาเลนไทน์' 'วันจักรี' 'สงกรานต์' 'วันแรงงาน (เอกชน)' 'วันฉัตรมงคล']
Valid data points: 0/2073 (0.0%)
❌ Skipped holiday_subject - insufficient valid data

Processing co

In [29]:
# ✅ แก้ไข: เตรียมข้อมูลปัจจัยภายนอกให้ครบถ้วน
def prepare_exog_variables(data):
    """เตรียม exogenous variables"""
    exog_data = pd.DataFrame(index=data.index)
    
    # ตรวจสอบและเตรียม day_type
    if 'day_type' in data.columns:
        # แทนที่ค่า missing ด้วย 0 (normal day)
        day_type_clean = data['day_type'].fillna(0)
        day_type_dummies = pd.get_dummies(day_type_clean, prefix='day_type')
        exog_data = pd.concat([exog_data, day_type_dummies], axis=1)
        print(f"✅ Added day_type dummies: {list(day_type_dummies.columns)}")
    else:
        # สร้าง dummy day_type ถ้าไม่มี
        exog_data['day_type_0'] = 1
        print("⚠️  No day_type found, created dummy variable")
    
    # ตรวจสอบและเตรียม rain_average
    if 'rain_average' in data.columns:
        # แทนที่ค่า missing ด้วย 0
        rain_clean = data['rain_average'].fillna(0)
        exog_data['rain_average'] = rain_clean
        print("✅ Added rain_average")
    else:
        # สร้าง dummy rain_average ถ้าไม่มี
        exog_data['rain_average'] = 0
        print("⚠️  No rain_average found, created dummy variable")
    
    # เติมค่า missing ที่เหลือ
    exog_data = exog_data.fillna(0)
    
    # ตรวจสอบว่าไม่มี inf หรือ nan
    exog_data = exog_data.replace([np.inf, -np.inf], 0)
    
    return exog_data

# เตรียม exogenous variables สำหรับ train และ test
print("Preparing exogenous variables...")
train_exog = prepare_exog_variables(train_df)
test_exog = prepare_exog_variables(test_df)

# ให้ test_exog มีคอลัมน์เดียวกันกับ train_exog
test_exog = test_exog.reindex(columns=train_exog.columns, fill_value=0)

print(f"\nExogenous variables: {list(train_exog.columns)}")
print(f"Train exog shape: {train_exog.shape}")
print(f"Test exog shape: {test_exog.shape}")
print(f"Missing values in train_exog: {train_exog.isnull().sum().sum()}")
print(f"Missing values in test_exog: {test_exog.isnull().sum().sum()}")


Preparing exogenous variables...
✅ Added day_type dummies: ['day_type_0', 'day_type_1', 'day_type_2']
✅ Added rain_average
✅ Added day_type dummies: ['day_type_0', 'day_type_1']
✅ Added rain_average

Exogenous variables: ['day_type_0', 'day_type_1', 'day_type_2', 'rain_average']
Train exog shape: (2043, 4)
Test exog shape: (30, 4)
Missing values in train_exog: 0
Missing values in test_exog: 0


In [30]:
# ✅ แก้ไข: สร้างฟังก์ชันคำนวณ MAPE ที่ robust
def calculate_mape(y_true, y_pred):
    """คำนวณ Mean Absolute Percentage Error แบบ robust"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # กรองข้อมูลที่ valid (ไม่ใช่ 0, nan หรือ inf)
    mask = (y_true != 0) & (~np.isnan(y_true)) & (~np.isnan(y_pred)) & \
           (~np.isinf(y_true)) & (~np.isinf(y_pred))
    
    if np.sum(mask) == 0:
        return np.nan
    
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ✅ แก้ไข: ฟังก์ชันหาพารามิเตอร์ที่เหมาะสมแบบง่าย
def find_simple_sarimax_params(series_length):
    """หาพารามิเตอร์ SARIMAX ที่เหมาะสมตามขนาดข้อมูล"""
    if series_length < 60:
        return (1, 1, 0), (0, 0, 0, 0)  # Simple ARIMA
    elif series_length < 200:
        return (1, 1, 1), (0, 0, 0, 0)  # ARIMA with MA
    else:
        return (2, 1, 1), (1, 0, 1, 7)  # SARIMAX with weekly pattern

# เก็บผลลัพธ์
forecast_results = {}
performance_metrics = {}

In [31]:
# แก้ไข cell ที่สร้างโมเดล

# วนลูปสำหรับแต่ละสายรถไฟฟ้า
for line in train_lines:
    print(f"\n{'='*50}")
    print(f"Processing {line}...")
    
    # ✅ แก้ไข: ตรวจสอบข้อมูลก่อนประมวลผล
    print(f"Data type: {train_df[line].dtype}")
    
    # ทำความสะอาดข้อมูลอีกครั้งเพื่อให้แน่ใจ
    train_series = train_df[line].dropna()
    test_series = test_df[line].dropna()
    
    print(f"Train series shape: {train_series.shape}")
    print(f"Test series shape: {test_series.shape}")
    print(f"Train series dtype: {train_series.dtype}")
    
    # ตรวจสอบว่าเป็นตัวเลขทั้งหมดหรือไม่
    if not pd.api.types.is_numeric_dtype(train_series):
        print(f"❌ Non-numeric data detected in {line}")
        print(f"Sample values: {train_series.head()}")
        continue
    
    if len(train_series) < 30:
        print(f"❌ Not enough data for {line} (only {len(train_series)} points)")
        continue
    
    # ตรวจสอบว่ามีความแปรปรวนหรือไม่
    if train_series.std() == 0:
        print(f"❌ No variation in {line}")
        continue
    
    try:
        # ✅ แก้ไข: ตรวจสอบ exog data ที่สอดคล้องกับ train_series
        exog_train = train_exog.loc[train_series.index]
        exog_test = test_exog
        
        print(f"Exog train shape: {exog_train.shape}")
        print(f"Exog test shape: {exog_test.shape}")
        
        # ตรวจสอบว่า exog data เป็นตัวเลขทั้งหมด
        if not exog_train.select_dtypes(include=[np.number]).shape[1] == exog_train.shape[1]:
            print(f"❌ Non-numeric exog data detected")
            continue
        
        # สร้างโมเดล SARIMAX
        model = SARIMAX(
            train_series,
            exog=exog_train,
            order=(2, 1, 1),  # (p,d,q)
            seasonal_order=(1, 1, 1, 7),  # (P,D,Q,s) - weekly seasonality
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        
        # Fit โมเดล
        print("Fitting model...")
        model_fit = model.fit(disp=False, maxiter=100)
        
        # ทำนาย 30 วันข้างหน้า
        print("Making forecast...")
        forecast_obj = model_fit.get_forecast(steps=len(test_series), exog=exog_test[:len(test_series)])
        forecast = forecast_obj.predicted_mean
        forecast_ci = forecast_obj.conf_int()
        
        # เก็บผลการทำนาย
        forecast_results[line] = {
            'forecast': forecast,
            'confidence_interval': forecast_ci,
            'model_fit': model_fit
        }
        
        # คำนวณ metrics เทียบกับข้อมูลจริง
        actual = test_series.values
        predicted = forecast.values
        
        # ✅ แก้ไข: ตรวจสอบขนาดข้อมูล
        min_len = min(len(actual), len(predicted))
        actual = actual[:min_len]
        predicted = predicted[:min_len]
        
        if len(actual) > 0:
            mae = mean_absolute_error(actual, predicted)
            mse = mean_squared_error(actual, predicted)
            rmse = np.sqrt(mse)
            mape = calculate_mape(actual, predicted)
            
            performance_metrics[line] = {
                'MAE': mae,
                'MSE': mse,
                'RMSE': rmse,
                'MAPE': mape,
                'Actual': actual,
                'Predicted': predicted,
                'Train_series': train_series,
                'Test_series': test_series,
                'Forecast': forecast
            }
            
            print(f"✅ MAE: {mae:.2f}")
            print(f"✅ RMSE: {rmse:.4f}")
            print(f"✅ MAPE: {mape:.2f}%")
        else:
            print(f"⚠️  No test data available for {line}")
        
    except Exception as e:
        print(f"❌ Error fitting model for {line}: {str(e)}")
        print(f"Train series sample: {train_series.head()}")
        print(f"Train series info: {train_series.info()}")
        continue

print(f"\n{'='*60}")
print("FORECAST RESULTS")
print(f"{'='*60}")

if forecast_results:
    for line, result in forecast_results.items():
        print(f"\n✅ Forecast for {line}:")
        forecast = result['forecast']
        for i, (date, value) in enumerate(zip(test_df.index[:len(forecast)], forecast), 1):
            print(f"  Day {i} ({date.strftime('%Y-%m-%d')}): {value:.0f}")
else:
    print("❌ No successful forecasts generated")


Processing รถไฟฟ้า ARL...
Data type: int64
Train series shape: (2043,)
Test series shape: (30,)
Train series dtype: int64
Exog train shape: (2043, 4)
Exog test shape: (30, 4)
❌ Non-numeric exog data detected

Processing รถไฟฟ้า BTS...
Data type: int64
Train series shape: (2043,)
Test series shape: (30,)
Train series dtype: int64
Exog train shape: (2043, 4)
Exog test shape: (30, 4)
❌ Non-numeric exog data detected

Processing รถไฟฟ้าสายสีน้ำเงิน...
Data type: int64
Train series shape: (2043,)
Test series shape: (30,)
Train series dtype: int64
Exog train shape: (2043, 4)
Exog test shape: (30, 4)
❌ Non-numeric exog data detected

Processing รถไฟฟ้าสายสีม่วง...
Data type: int64
Train series shape: (2043,)
Test series shape: (30,)
Train series dtype: int64
Exog train shape: (2043, 4)
Exog test shape: (30, 4)
❌ Non-numeric exog data detected

Processing รถไฟฟ้าสายสีแดง...
Data type: float64
Train series shape: (1464,)
Test series shape: (30,)
Train series dtype: float64
Exog train shape: (1

In [32]:
# ✅ แก้ไข: แก้ปัญหาการ fit โมเดล

# วนลูปสำหรับแต่ละสายรถไฟฟ้า
for line in train_lines:
    print(f"\n{'='*50}")
    print(f"Processing {line}...")
    
    try:
        # เตรียมข้อมูลสำหรับสายนี้
        train_series = train_df[line].dropna()
        test_series = test_df[line].dropna()
        
        print(f"Train series length: {len(train_series)}")
        print(f"Test series length: {len(test_series)}")
        print(f"Data type: {train_series.dtype}")
        
        # ตรวจสอบข้อมูลเบื้องต้น
        if len(train_series) < 10:
            print(f"❌ Not enough data for {line} (only {len(train_series)} points)")
            continue
        
        if train_series.std() == 0:
            print(f"❌ No variation in {line}")
            continue
        
        # ตรวจสอบค่า extreme values
        if (train_series < 0).any():
            print(f"⚠️  Negative values found, setting to 0")
            train_series = train_series.clip(lower=0)
        
        # เตรียม exogenous data ให้ match กับ train_series
        exog_train = train_exog.loc[train_series.index].copy()
        
        # ตรวจสอบ exog data
        if exog_train.isnull().any().any():
            print(f"⚠️  Missing values in exog data, filling with 0")
            exog_train = exog_train.fillna(0)
        
        print(f"Exog train shape: {exog_train.shape}")
        
        # หาพารามิเตอร์ที่เหมาะสม
        order, seasonal_order = find_simple_sarimax_params(len(train_series))
        print(f"Using parameters: order={order}, seasonal_order={seasonal_order}")
        
        # ลองสร้างโมเดลแบบง่ายก่อน (ไม่ใช้ exog)
        try:
            print("Trying simple ARIMA model first...")
            simple_model = SARIMAX(
                train_series,
                order=order,
                seasonal_order=(0, 0, 0, 0),  # ไม่ใช้ seasonal
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            simple_fit = simple_model.fit(disp=False, maxiter=50)
            print("✅ Simple ARIMA model fitted successfully")
            
            # ทำนายด้วยโมเดลง่าย
            simple_forecast = simple_fit.get_forecast(steps=min(30, len(test_series)))
            forecast = simple_forecast.predicted_mean
            forecast_ci = simple_forecast.conf_int()
            
        except Exception as e:
            print(f"❌ Simple ARIMA failed: {e}")
            print("Trying even simpler model...")
            
            # ลองโมเดลง่ายที่สุด
            try:
                simplest_model = SARIMAX(
                    train_series,
                    order=(1, 0, 0),
                    enforce_stationarity=False,
                    enforce_invertibility=False
                )
                simple_fit = simplest_model.fit(disp=False, maxiter=30)
                simple_forecast = simple_fit.get_forecast(steps=min(30, len(test_series)))
                forecast = simple_forecast.predicted_mean
                forecast_ci = simple_forecast.conf_int()
                print("✅ Simplest AR(1) model fitted")
                
            except Exception as e2:
                print(f"❌ All models failed for {line}: {e2}")
                continue
        
        # เก็บผลการทำนาย
        forecast_results[line] = {
            'forecast': forecast,
            'confidence_interval': forecast_ci,
            'model_fit': simple_fit
        }
        
        # คำนวณ metrics เทียบกับข้อมูลจริง
        if len(test_series) > 0:
            actual = test_series.values
            predicted = forecast.values
            
            # ปรับขนาดให้เท่ากัน
            min_len = min(len(actual), len(predicted))
            actual = actual[:min_len]
            predicted = predicted[:min_len]
            
            if min_len > 0:
                mae = mean_absolute_error(actual, predicted)
                mse = mean_squared_error(actual, predicted)
                rmse = np.sqrt(mse)
                mape = calculate_mape(actual, predicted)
                
                performance_metrics[line] = {
                    'MAE': mae,
                    'MSE': mse,
                    'RMSE': rmse,
                    'MAPE': mape,
                    'Actual': actual,
                    'Predicted': predicted,
                    'Train_series': train_series,
                    'Test_series': test_series,
                    'Forecast': forecast
                }
                
                print(f"✅ MAE: {mae:.2f}")
                print(f"✅ RMSE: {rmse:.2f}")
                if not np.isnan(mape):
                    print(f"✅ MAPE: {mape:.2f}%")
                else:
                    print("⚠️  MAPE: Cannot calculate (division by zero)")
            else:
                print(f"⚠️  No matching test data for evaluation")
        else:
            print(f"⚠️  No test data available for {line}")
        
    except Exception as e:
        print(f"❌ Error processing {line}: {str(e)}")
        import traceback
        print(f"Details: {traceback.format_exc()}")
        continue

print(f"\n{'='*60}")
print("FORECAST RESULTS")
print(f"{'='*60}")

if forecast_results:
    for line, result in forecast_results.items():
        print(f"\n✅ Forecast for {line}:")
        forecast = result['forecast']
        # ใช้ test_df index สำหรับวันที่
        forecast_dates = test_df.index[:len(forecast)]
        for i, (date, value) in enumerate(zip(forecast_dates, forecast), 1):
            print(f"  Day {i} ({date.strftime('%Y-%m-%d')}): {value:.0f}")
else:
    print("❌ No successful forecasts generated")


Processing รถไฟฟ้า ARL...
Train series length: 2043
Test series length: 30
Data type: int64
Exog train shape: (2043, 4)
Using parameters: order=(2, 1, 1), seasonal_order=(1, 0, 1, 7)
Trying simple ARIMA model first...
✅ Simple ARIMA model fitted successfully
✅ MAE: 11113.97
✅ RMSE: 11758.11
✅ MAPE: 17.67%

Processing รถไฟฟ้า BTS...
Train series length: 2043
Test series length: 30
Data type: int64
Exog train shape: (2043, 4)
Using parameters: order=(2, 1, 1), seasonal_order=(1, 0, 1, 7)
Trying simple ARIMA model first...
✅ Simple ARIMA model fitted successfully
✅ MAE: 11113.97
✅ RMSE: 11758.11
✅ MAPE: 17.67%

Processing รถไฟฟ้า BTS...
Train series length: 2043
Test series length: 30
Data type: int64
Exog train shape: (2043, 4)
Using parameters: order=(2, 1, 1), seasonal_order=(1, 0, 1, 7)
Trying simple ARIMA model first...
✅ Simple ARIMA model fitted successfully
✅ MAE: 135475.15
✅ RMSE: 145198.53
✅ MAPE: 19.99%

Processing รถไฟฟ้าสายสีน้ำเงิน...
Train series length: 2043
Test series l

In [33]:
# ✅ แสดงสรุปผลลัพธ์และสถิติ

print(f"\n{'='*60}")
print("PERFORMANCE SUMMARY")
print(f"{'='*60}")

if performance_metrics:
    # สร้าง DataFrame สำหรับสรุปผลลัพธ์
    summary_data = []
    for line, metrics in performance_metrics.items():
        summary_data.append({
            'Line': line,
            'MAE': metrics['MAE'],
            'RMSE': metrics['RMSE'],
            'MAPE': metrics['MAPE'] if not np.isnan(metrics['MAPE']) else 'N/A',
            'Train_Days': len(metrics['Train_series']),
            'Test_Days': len(metrics['Actual'])
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    # หาโมเดลที่ดีที่สุด
    valid_mapes = [m['MAPE'] for m in performance_metrics.values() if not np.isnan(m['MAPE'])]
    if valid_mapes:
        best_line = min(performance_metrics.keys(), 
                       key=lambda x: performance_metrics[x]['MAPE'] if not np.isnan(performance_metrics[x]['MAPE']) else float('inf'))
        print(f"\n🏆 Best performing model: {best_line}")
        print(f"   MAPE: {performance_metrics[best_line]['MAPE']:.2f}%")
    
else:
    print("❌ No performance metrics available")

print(f"\n{'='*60}")
print("MODEL STATUS")
print(f"{'='*60}")
print(f"✅ Successfully fitted models: {len(forecast_results)}")
print(f"❌ Failed models: {len(train_lines) - len(forecast_results)}")
print(f"📊 Total train lines processed: {len(train_lines)}")


PERFORMANCE SUMMARY
               Line           MAE          RMSE      MAPE  Train_Days  Test_Days
        รถไฟฟ้า ARL  11113.966836  11758.109091 17.671540        2043         30
        รถไฟฟ้า BTS 135475.147206 145198.529754 19.991404        2043         30
รถไฟฟ้าสายสีน้ำเงิน 102971.191926 107396.981581 26.309905        2043         30
   รถไฟฟ้าสายสีม่วง  19352.433839  20895.021349 34.148411        2043         30
    รถไฟฟ้าสายสีแดง   7338.035868   7684.275107 20.451054        1464         30

🏆 Best performing model: รถไฟฟ้า ARL
   MAPE: 17.67%

MODEL STATUS
✅ Successfully fitted models: 5
❌ Failed models: 0
📊 Total train lines processed: 5
